In [1]:
import numpy as np
from numpy import cumsum

num_dogs = 30
num_trails = 25


In [2]:
y = np.loadtxt('data.txt')
y_failure = 1 - y

xa = cumsum(y,axis=1) - y
xs = cumsum(y_failure,axis=1) - y_failure
print xa[0]

[  0.   0.   0.   1.   1.   2.   2.   3.   4.   5.   6.   7.   8.   9.  10.
  11.  12.  13.  14.  15.  16.  17.  18.  19.  20.]


In [3]:
alpha = -1
beta = -1


In [4]:
from scipy.stats import expon
from scipy.stats import norm
def likelihood(alpha,beta,data):
    p_log = np.zeros((30, 25), dtype=np.float64)
    p = np.zeros((30, 25), dtype=np.float64)
    p_log = alpha * xa + beta * xs
    p = np.exp(p_log)
   
    prob = np.zeros((30, 25), dtype=np.float64)
    for d in range(30):
        for t in range(25):
            if data[d][t] == 0: 
                prob[d][t] = p[d][t]
            else:
                prob[d][t] = 1 - p[d][t]

    likelihood = prob.prod()
    
    return likelihood
    

def prior(alpha,beta):
    alpha_prior = norm.pdf(alpha)
    beta_prior = norm.pdf(beta)
    
    return alpha_prior*beta_prior

def proposal_prob(alpha,beta):
    alpha_prior = norm.pdf(alpha,scale=0.27)
    beta_prior = norm.pdf(beta,scale=0.27)
    
    return alpha_prior*beta_prior

def target_rel_prob(alpha,beta, data):
    
    target_rel_prob = likelihood(alpha,beta,data) * prior(alpha,beta)
   
    return target_rel_prob



In [5]:
#MCMC
traj_length = 10000 # arbitrary large number
# Initialize the vector that will store the results:

# Specify where to start the trajectory:
posterior_alpha = [alpha] # arbitrary value
posterior_beta = [beta]
#posterior = [target_rel_prob(alpha,beta,y_failure)]
# Specify the burn-in period:
burn_in = 100 # arbitrary number, less than traj_length
burn_in = int(burn_in)

# Initialize accepted, rejected counters, just to monitor performance:
n_accepted = 0
n_rejected = 0
# Specify seed to reproduce same random walk:
np.random.seed(4745)
data = y


current_alpha = alpha
current_beta = beta
#current_posterior = target_rel_prob(alpha,beta,y_failure)
for t in range(traj_length):
    while True:
        proposed_jump_alpha = norm.rvs(current_alpha,scale=0.5)
        if proposed_jump_alpha<-0.00001:
            break;
    while True:
        proposed_jump_beta = norm.rvs(current_beta,scale=0.5)
        if proposed_jump_beta<-0.00001:
            break;
    
    #prob_accept = np.minimum(1,(target_rel_prob(proposed_jump_alpha,proposed_jump_beta,data)*proposal_prob(current_alpha,current_beta))/
    #                        (target_rel_prob(current_alpha,current_beta,data)*proposal_prob(proposed_jump_alpha,proposed_jump_beta)))
      
    prob_accept = np.minimum(1,(target_rel_prob(proposed_jump_alpha,proposed_jump_beta,data))/
                            (target_rel_prob(current_alpha,current_beta,data)))  
        # increment the accepted counter, just to monitor performance
    if t > burn_in:
        
        if np.random.rand() <= prob_accept:
            
            n_accepted += 1
            posterior_alpha.append(proposed_jump_alpha)
            posterior_beta.append(proposed_jump_beta)
           
            current_alpha = proposed_jump_alpha
            current_beta = proposed_jump_beta
          
        else:
            n_rejected += 1
            posterior_alpha.append(current_alpha)
            posterior_beta.append(current_alpha)
           
    else:
            n_rejected += 1

print 'accepted',n_accepted
print n_rejected

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:38: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:38: RuntimeWarning: divide by zero encountered in double_scalars


accepted 52
9948


In [6]:
# Extract the post-burn_in portion of the trajectory.
accepted_traj = np.column_stack((posterior_alpha,posterior_beta))
# End of Metropolis algorithm.
avg_alpha=0 
avg_beta=0 
for i in range(len(posterior_alpha)):
    avg_alpha = avg_alpha+posterior_alpha[i]
    avg_beta = avg_beta+posterior_beta[i]
alpha_mean = avg_alpha/(len(posterior_alpha))
beta_mean = avg_beta/(len(posterior_beta))


In [16]:

x_shock=8;
x_avoidance=17;

sum_prob=0
for j in range(len(posterior_alpha)):
    log_p = (posterior_alpha[j] * x_avoidance + posterior_beta[j] * x_shock)
    prob_shock = np.exp(log_p)
    sum_prob = sum_prob + prob_shock
sum_prob = sum_prob/len(posterior_alpha)    
print sum_prob
if(sum_prob>0.5):
    print 'Dog gets a shock in the next trial'
        
else:
     print 'Dog avoids shock in the next trial'
print 'alpha mean = ',alpha_mean
print 'beta mean = ',beta_mean

0.00245735132818
Dog avoids shock in the next trial
alpha mean =  -0.251300720123
beta mean =  -0.250321659862
